# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff3ec317bb0>
├── 'a' --> tensor([[ 0.9014, -0.6031,  1.1265],
│                   [ 0.9373,  0.4985, -0.3968]])
└── 'x' --> <FastTreeValue 0x7ff3ec3178b0>
    └── 'c' --> tensor([[-0.6597, -0.3851,  0.0208, -0.7360],
                        [-0.8712, -0.8563,  1.4484, -0.1265],
                        [ 1.3824,  0.0637,  0.9680,  1.1617]])

In [4]:
t.a

tensor([[ 0.9014, -0.6031,  1.1265],
        [ 0.9373,  0.4985, -0.3968]])

In [5]:
%timeit t.a

65.5 ns ± 0.0285 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff3ec317bb0>
├── 'a' --> tensor([[ 0.3659, -0.5708,  0.3303],
│                   [ 0.8927,  0.3929,  0.5440]])
└── 'x' --> <FastTreeValue 0x7ff3ec3178b0>
    └── 'c' --> tensor([[-0.6597, -0.3851,  0.0208, -0.7360],
                        [-0.8712, -0.8563,  1.4484, -0.1265],
                        [ 1.3824,  0.0637,  0.9680,  1.1617]])

In [7]:
%timeit t.a = new_value

68.3 ns ± 0.0467 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9014, -0.6031,  1.1265],
               [ 0.9373,  0.4985, -0.3968]]),
    x: Batch(
           c: tensor([[-0.6597, -0.3851,  0.0208, -0.7360],
                      [-0.8712, -0.8563,  1.4484, -0.1265],
                      [ 1.3824,  0.0637,  0.9680,  1.1617]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9014, -0.6031,  1.1265],
        [ 0.9373,  0.4985, -0.3968]])

In [11]:
%timeit b.a

60.5 ns ± 0.113 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0843, -0.6908, -0.0216],
               [-1.1765,  0.9935, -0.1194]]),
    x: Batch(
           c: tensor([[-0.6597, -0.3851,  0.0208, -0.7360],
                      [-0.8712, -0.8563,  1.4484, -0.1265],
                      [ 1.3824,  0.0637,  0.9680,  1.1617]]),
       ),
)

In [13]:
%timeit b.a = new_value

496 ns ± 0.0912 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

820 ns ± 3.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 49.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 506 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 1.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff3ec37aca0>
├── 'a' --> tensor([[[ 0.9014, -0.6031,  1.1265],
│                    [ 0.9373,  0.4985, -0.3968]],
│           
│                   [[ 0.9014, -0.6031,  1.1265],
│                    [ 0.9373,  0.4985, -0.3968]],
│           
│                   [[ 0.9014, -0.6031,  1.1265],
│                    [ 0.9373,  0.4985, -0.3968]],
│           
│                   [[ 0.9014, -0.6031,  1.1265],
│                    [ 0.9373,  0.4985, -0.3968]],
│           
│                   [[ 0.9014, -0.6031,  1.1265],
│                    [ 0.9373,  0.4985, -0.3968]],
│           
│                   [[ 0.9014, -0.6031,  1.1265],
│                    [ 0.9373,  0.4985, -0.3968]],
│           
│                   [[ 0.9014, -0.6031,  1.1265],
│                    [ 0.9373,  0.4985, -0.3968]],
│           
│                   [[ 0.9014, -0.6031,  1.1265],
│                    [ 0.9373,  0.4985, -0.3968]]])
└── 'x' --> <FastTreeValue 0x7ff3ec37ae50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.4 µs ± 73.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff340fd0610>
├── 'a' --> tensor([[ 0.9014, -0.6031,  1.1265],
│                   [ 0.9373,  0.4985, -0.3968],
│                   [ 0.9014, -0.6031,  1.1265],
│                   [ 0.9373,  0.4985, -0.3968],
│                   [ 0.9014, -0.6031,  1.1265],
│                   [ 0.9373,  0.4985, -0.3968],
│                   [ 0.9014, -0.6031,  1.1265],
│                   [ 0.9373,  0.4985, -0.3968],
│                   [ 0.9014, -0.6031,  1.1265],
│                   [ 0.9373,  0.4985, -0.3968],
│                   [ 0.9014, -0.6031,  1.1265],
│                   [ 0.9373,  0.4985, -0.3968],
│                   [ 0.9014, -0.6031,  1.1265],
│                   [ 0.9373,  0.4985, -0.3968],
│                   [ 0.9014, -0.6031,  1.1265],
│                   [ 0.9373,  0.4985, -0.3968]])
└── 'x' --> <FastTreeValue 0x7ff340febf40>
    └── 'c' --> tensor([[-0.6597, -0.3851,  0.0208, -0.7360],
                        [-0.8712, -0.8563,  1.4484, -0.1265],
                 

In [23]:
%timeit t_cat(trees)

29.6 µs ± 74 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.8 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.6597, -0.3851,  0.0208, -0.7360],
                       [-0.8712, -0.8563,  1.4484, -0.1265],
                       [ 1.3824,  0.0637,  0.9680,  1.1617]],
              
                      [[-0.6597, -0.3851,  0.0208, -0.7360],
                       [-0.8712, -0.8563,  1.4484, -0.1265],
                       [ 1.3824,  0.0637,  0.9680,  1.1617]],
              
                      [[-0.6597, -0.3851,  0.0208, -0.7360],
                       [-0.8712, -0.8563,  1.4484, -0.1265],
                       [ 1.3824,  0.0637,  0.9680,  1.1617]],
              
                      [[-0.6597, -0.3851,  0.0208, -0.7360],
                       [-0.8712, -0.8563,  1.4484, -0.1265],
                       [ 1.3824,  0.0637,  0.9680,  1.1617]],
              
                      [[-0.6597, -0.3851,  0.0208, -0.7360],
                       [-0.8712, -0.8563,  1.4484, -0.1265],
                       [ 1.3824,  0.0637,  0.9680,  1.1617]],

In [26]:
%timeit Batch.stack(batches)

79.4 µs ± 269 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.6597, -0.3851,  0.0208, -0.7360],
                      [-0.8712, -0.8563,  1.4484, -0.1265],
                      [ 1.3824,  0.0637,  0.9680,  1.1617],
                      [-0.6597, -0.3851,  0.0208, -0.7360],
                      [-0.8712, -0.8563,  1.4484, -0.1265],
                      [ 1.3824,  0.0637,  0.9680,  1.1617],
                      [-0.6597, -0.3851,  0.0208, -0.7360],
                      [-0.8712, -0.8563,  1.4484, -0.1265],
                      [ 1.3824,  0.0637,  0.9680,  1.1617],
                      [-0.6597, -0.3851,  0.0208, -0.7360],
                      [-0.8712, -0.8563,  1.4484, -0.1265],
                      [ 1.3824,  0.0637,  0.9680,  1.1617],
                      [-0.6597, -0.3851,  0.0208, -0.7360],
                      [-0.8712, -0.8563,  1.4484, -0.1265],
                      [ 1.3824,  0.0637,  0.9680,  1.1617],
                      [-0.6597, -0.3851,  0.0208, -0.7360],
                   

In [28]:
%timeit Batch.cat(batches)

145 µs ± 608 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

338 µs ± 2.48 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
